In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as DF
from sympy import diff
from sympy.abc import x,y
from sympy import cos
from sympy import *
from sympy import lambdify
from itertools import product
import random
from collections import defaultdict
import copy
import time

**Define path**

In [2]:
# Defining path to folder raw data is saved in
path = "C:/Users/robtk/OneDrive/Desktop/DIAS Internship/Raw data/Harmonic oscillator warm up/"

### Set display of pandas dataframes and Series to 11 decimal places

In [3]:
pd.set_option("display.precision", 11)

**Define the commutator**

In [4]:
def commutator(A,B):
    
    # matmul() multiplies two matrices together
    return np.matmul(A,B) - np.matmul(B,A)

In [5]:
M1 = np.array([[1,2,3], [4,5,6], [7,8,9]])
M2 = np.array([[8,7,6], [5,4,3], [3,2,1]])
M3 = np.array([[3,4,2],[5,6,7],[4,3,2]])
M4 = np.array([[2,1,2], [1,2,1], [2,1,2]])

In [6]:
commutator(M2, commutator(M1, M2))

array([[ 1566,   747,   -72],
       [  -39,  -354,  -669],
       [-1332, -1272, -1212]])

In [7]:
commutator(M3, commutator(M1, M3))

array([[ 267,  170,  -50],
       [ 774,  608,  157],
       [-630, -808, -875]])

In [8]:
commutator(M2, commutator(M1, M2)) + commutator(M3, commutator(M1, M3)) + commutator(M1, commutator(M1, M1)) + commutator(M4, commutator(M1, M4))

array([[ 1939,   961,   -64],
       [  743,   234,  -540],
       [-2000, -2144, -2173]])

### Define Lagrangian

In [9]:
def L(X_list, V_list, g):
    
    X_array = np.array(X_list)
    V_array = np.array(X_list)
    term_1 =  np.sum( np.trace( (V_array)**2 ) )
    
    # Calculate the sum of commutators found in the second term of the lagrangian
    for el in X_list:
        X_cut = [x for x in X_list if x != el]
        Sum2 = np.sum( np.trace( (commutator(el,X_cut))**2 ) ) 
    
    term_2 = 1/2 * Sum2
    
    return 1/(2*g**2) * (term_1 + term_2)

**Define the Hamiltonian**

In [10]:
def H(X_list, V_list, g):
    
    V_array = np.array(X_list)
    V_array_T = np.transpose(V_array)
    
    return sum( V_array_T * V_array /g**2) - L(X_list, V_list, g)
    

Initial reference point is x_0 = (X, V)

**Define the acceleration equation of motion as gotten from the research paper.**

In [11]:
def acceleration(X_list):
    
    # This function creates a list of all the accelerations of all the coordinates from a list of the coordinate matrices.
    
    # List to be filled with accelerations for the coordinates
    A_list = []
    
    # Turn the coordinates into an array so all X^j can put into the equation of motion in one line.
    array = np.array(X_list)
    
    # for loop to calculate all the coordinate accelerations.
    for el in X_list:
        
        # Doing the equation of motion
        # el == X^i and array == X^j in the equation of motion
        equation_of_motion = commutator(array ,commutator(el, array))
        
        # sum() carries out the summation symbol in the equation of motion and so the acceleration is calculated.
        # append the acceleration to the list.
        A_list.append(sum(equation_of_motion))
        
    return A_list

**Code for evolving the initial point in phase space for 1000 seconds so that the system is thermalised.**

In [12]:
def thermalising_algorithm(X_list, V_list, A_list, delta_t):
    
    # Define initial conditions as numpy arrays
    X_array_old = np.array(X_list)
    V_array_old = np.array(V_list)
    A_array_old = np.array(A_list)
    
    i = 0
    # Repeating algorithm for 1000 seconds to thermalise the system.
    
    while i < 100/delta_t:
        
        # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
        
        X_array_new = (X_array_old + V_array_old * delta_t + 1/2 * A_array_old * delta_t**2)

        
        # c) Use equations of motion to get new acceleration values
        
        A_array_new = np.array(acceleration(X_array_new.tolist()))

        # d) Use Velocity Verlet 2 to get new momentums

        V_array_new = (V_array_old + (1/2) * (A_array_new + A_array_old) * delta_t)
                
        # Update coordinates
        X_array_old = X_array_new
        V_array_old = V_array_new
        A_array_old = A_array_new
        
        if i % 1000 == True:
            print("Thermal", i, X_array_new)

        
        i += 1
        
    return X_array_new, V_array_new, A_array_new

**Code for evolving the simulation and recording the position and momentum after it was thermalised.**

In [22]:
def evolving_algorithm(X_list, V_list, A_list, delta_t, record_steps, simulation_repetitions, 
                       X_labels, V_labels, N, times):
    
    i = 1
    
    # Define initial conditions as numpy arrays
    X_array_old = np.array(X_list)
    V_array_old = np.array(V_list)
    A_array_old = np.array(A_list)
    
    # Coordinates dictionaries
    recorded_X1 = { "0" : X_list[0].tolist()}
    recorded_X2 = { "0" : X_list[1].tolist()}
    recorded_V1 = { "0" : V_list[0].tolist()}
    recorded_V2 = { "0" : V_list[1].tolist()}
    recorded_X3 = { "0" : X_list[2].tolist()}
    recorded_X4 = { "0" : X_list[3].tolist()}
    recorded_V3 = { "0" : V_list[2].tolist()}
    recorded_V4 = { "0" : V_list[3].tolist()}
    recorded_X5 = { "0" : X_list[4].tolist()}
    recorded_X6 = { "0" : X_list[5].tolist()}
    recorded_V5 = { "0" : V_list[4].tolist()}
    recorded_V6 = { "0" : V_list[5].tolist()}
    recorded_X7 = { "0" : X_list[6].tolist()}
    recorded_X8 = { "0" : X_list[7].tolist()}
    recorded_V7 = { "0" : V_list[6].tolist()}
    recorded_V8 = { "0" : V_list[7].tolist()}
    recorded_X9 = { "0" : X_list[8].tolist()}
    recorded_V9 = { "0" : X_list[8].tolist()}



    # Repeating algorithm
    while i < simulation_repetitions:
        
        # b) velocity Verlet 1 to get new positions from old positions, momentums and rate of change of momentums
        
        X_array_new = (X_array_old + V_array_old * delta_t + 1/2 * A_array_old * delta_t**2)

        
        # c) Use equations of motion to get new acceleration values
        
        A_array_new = np.array(acceleration(X_array_new.tolist()))


        # d) Use Velocity Verlet 2 to get new momentums

        V_array_new = (V_array_old + (1/2) * (A_array_new + A_array_old) * delta_t)
        
        
        # Recording position, momentum, and rate of change of momentum every 10th step
        if i % record_steps == 0:

            recorded_X1[f"{delta_t*record_steps*i}"] = X_array_new[0].tolist()
            recorded_X2[f"{delta_t*record_steps*i}"] = X_array_new[1].tolist()
            recorded_V1[f"{delta_t*record_steps*i}"] = V_array_new[0].tolist()
            recorded_V2[f"{delta_t*record_steps*i}"] = V_array_new[1].tolist()
            
            recorded_X3[f"{delta_t*record_steps*i}"] = X_array_new[2].tolist()
            recorded_X4[f"{delta_t*record_steps*i}"] = X_array_new[3].tolist()
            recorded_V3[f"{delta_t*record_steps*i}"] = V_array_new[2].tolist()
            recorded_V4[f"{delta_t*record_steps*i}"] = V_array_new[3].tolist()
            
            recorded_X5[f"{delta_t*record_steps*i}"] = X_array_new[4].tolist()
            recorded_X6[f"{delta_t*record_steps*i}"] = X_array_new[5].tolist()
            recorded_V5[f"{delta_t*record_steps*i}"] = V_array_new[4].tolist()
            recorded_V6[f"{delta_t*record_steps*i}"] = V_array_new[5].tolist()
            
            recorded_X7[f"{delta_t*record_steps*i}"] = X_array_new[6].tolist()
            recorded_X8[f"{delta_t*record_steps*i}"] = X_array_new[7].tolist()
            recorded_V7[f"{delta_t*record_steps*i}"] = V_array_new[6].tolist()
            recorded_V8[f"{delta_t*record_steps*i}"] = V_array_new[7].tolist()
            
            recorded_X9[f"{delta_t*record_steps*i}"] = X_array_new[8].tolist()
            recorded_V9[f"{delta_t*record_steps*i}"] = V_array_new[8].tolist()

            

        
            
        # Update coordinates
        X_array_old = X_array_new
        V_array_old = V_array_new
        A_array_old = A_array_new
         

        i += 1
    
    return recorded_X1, recorded_X2, recorded_V1, recorded_V2, recorded_X3, recorded_X4, recorded_V3, recorded_V4, recorded_X5, recorded_X6, recorded_V5, recorded_V6, recorded_X7, recorded_X8, recorded_V7, recorded_V8, recorded_X9, recorded_V9


**Create initial position matrices: Take X^i to have random (Gaussian) matrix elements, and be traceless.**

In [23]:
def simulation(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))
    print(X_list)
        
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < dim:
        V_list.append(V_initial)
        i += 1
    print(V_list)
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)
    

    # Define number of steps between measurement recordings
    record_steps = 100
    


    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 1000000
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.
 
    thermalised_X_array, thermalised_V_array, thermalised_A_array = thermalising_algorithm(X_list, V_list, A_list, delta_t)
        
    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Now that the system is thermalised and the thermalised initial conditions are found, evolve the system
    # and run the data collecting simulation.
    recorded_X1, recorded_X2, recorded_V1, recorded_V2 = evolving_algorithm(thermalised_X_array,
                                                                            thermalised_V_array, 
                                                                            thermalised_A_array,
                                          delta_t, record_steps, simulation_repetitions, 
                                          X_labels, V_labels, N, times)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T

    X = pd.concat([X1, X2], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2"})
    
    V = pd.concat([V1, V2], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2"})
    

    
    return X, V

In [24]:
def simulation_no_thermalising(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, .01, size = (dim, N, N))
    print(X_list)
        
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < dim:
        V_list.append(V_initial)
        i += 1
    print(V_list)
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)
    

    # Define number of steps between measurement recordings
    record_steps = 100
    


    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 200000
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.

    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Now that the system is thermalised and the thermalised initial conditions are found, evolve the system
    # and run the data collecting simulation.
    recorded_X1, recorded_X2, recorded_V1, recorded_V2, recorded_X3, recorded_X4, recorded_V3, recorded_V4, recorded_X5, recorded_X6, recorded_V5, recorded_V6, recorded_X7, recorded_X8, recorded_V7, recorded_V8, recorded_X9, recorded_V9 = evolving_algorithm(X_list,
                                                                            V_list, 
                                                                            A_list,
                                          delta_t, record_steps, simulation_repetitions, 
                                          X_labels, V_labels, N, times)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T
    
    X3 = pd.Series(recorded_X3).T
    X4 = pd.Series(recorded_X4).T
    V3 = pd.Series(recorded_V3).T
    V4 = pd.Series(recorded_V4).T
    
    X5 = pd.Series(recorded_X5).T
    X6 = pd.Series(recorded_X6).T
    V5 = pd.Series(recorded_V5).T
    V6 = pd.Series(recorded_V6).T
    
    X7 = pd.Series(recorded_X7).T
    X8 = pd.Series(recorded_X8).T
    V7 = pd.Series(recorded_V7).T
    V8 = pd.Series(recorded_V8).T
    
    X9 = pd.Series(recorded_X9).T
    V9 = pd.Series(recorded_V9).T


    X = pd.concat([X1, X2, X3, X4, X5, X6, X7, X8, X9], axis = 1)
    X = X.rename(columns = {0:"X1", 1:"X2", 2 : "X3", 3 : "X4", 4 : "X5", 5 : "X6", 6 : "X7", 7 : "X8", 8: "X9"})
    
    V = pd.concat([V1, V2,  X3, X4, X5, X6, X7, X8, X9], axis = 1)
    V = V.rename(columns = {0:"V1", 1:"V2", 2 : "V3", 3 : "V4", 4 : "V5", 5 : "V6", 6 : "V7", 7 : "V8" , 8 : "V9" })
    

    
    return X, V

In [25]:
start = time.time()
X, V= simulation_no_thermalising(9, 2, 10**-4)

print(time.time()-start)

[[[ 0.00852678 -0.02062536]
  [-0.00592257 -0.00090242]]

 [[-0.00110882  0.01211135]
  [-0.00013425 -0.01309701]]

 [[-0.00485054 -0.00910074]
  [ 0.00303395 -0.00620177]]

 [[-0.00836534 -0.00231308]
  [ 0.00275088  0.00415804]]

 [[-0.0204797   0.00441133]
  [ 0.00261109  0.00633968]]

 [[-0.00254534  0.00751372]
  [-0.00224456 -0.00063539]]

 [[ 0.01797322  0.00184433]
  [ 0.00596583 -0.00980074]]

 [[-0.00280608  0.00251675]
  [ 0.0054131  -0.00440859]]

 [[-0.00070134 -0.00973625]
  [ 0.00308994 -0.00265466]]]
[array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]])]
73.88971662521362


In [36]:
X.iloc[1999, :]

X1    [[0.008280329021426421, -0.014687391181263026]...
X2    [[-0.0014247583422862097, 0.007274605034940272...
X3    [[-0.005001737589217169, -0.004934400658866044...
X4    [[-0.008050136150821494, -0.000439409162013051...
X5    [[-0.019688580110588445, 0.004063234478728954]...
X6    [[-0.0024018397831547314, 0.004160457225775887...
X7    [[0.016982950415632345, 0.002083108045623025],...
X8    [[-0.0029690221284731718, 0.002959746907274272...
X9    [[-0.0008752748498037435, -0.00533152467009855...
Name: 1999.0, dtype: object

### Set display of pandas dataframes and Series to 11 decimal places

In [27]:
pd.set_option("display.precision", 11)

In [28]:
X.to_pickle(path+"X_test10.pkl")
V.to_pickle(path+"V_test10.pkl")

# Lyapunov exponent

1. Thermalise system
1. Perturb system
1. Evolve both original and perturbed system, measuring the exponential rate at which they diverge.

In [ ]:
def simulation_2_Points(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    X_list = np.random.normal(0, 1, size = (dim, N, N))
    
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < len(X_list):
        V_list.append(V_initial)
        i += 1
    
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(X_list)

    # Define number of steps between measurement recordings
    record_steps = 100000
    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 10000000
    
    
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.
    thermalised_X_array, thermalised_V_array, thermalised_A_array = thermalising_algorithm(X_list, V_list, A_list, delta_t)
        
    
    #### New Code ####
    
    # Perturb the thermalised_initial coordinates to get the second system
    
    
    
        
    ####          #### 
    
    # Labels for coordinates dictionary
    X_labels = []
    j = 1
    while j <= dim:
        X_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    X1 = pd.Series(recorded_X1).T
    X2 = pd.Series(recorded_X2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T

    X = pd.concat([X1, X2], axis = 1)
    X = X.rename(columns = {0:"X 1", 1:"X 2"})
    
    V = pd.concat([V1, V2], axis = 1)
    V = V.rename(columns = {0:"V 1", 1:"V 2"})
    

    
    return X, V

**Sub X = gY**

In [ ]:
def simulation_no_thermalising(dim, N, delta_t):
    # N is the length of the side of the individual position square matrices.
    # dim is the number of position coordinates we need.
    
    # Create position coordinate matrices with gaussian distributed elements.
    
    Y_list = np.random.normal(0, .01, size = (dim, N, N))
    print(Y_list)
        
    # The velocity Verlet algorithm conserves Gauss' law, so if it starts off true, it remains true. 
    # So just ensure the initial V^i are zero matrices.  

    V_initial = np.zeros((N, N))
    i = 0
    V_list = []
    while i < dim:
        V_list.append(V_initial)
        i += 1
    print(V_list)
    # Use the scalar equation of motion in our gauge to calculate the initial accelerations from the initial positions
    
    A_list = acceleration(Y_list)
    

    # Define number of steps between measurement recordings
    record_steps = 100
    


    
    # How many iterations the simulation will go on for after it has been thermalised.
    simulation_repetitions = 200000
    # Thermalise the system. , i.e. evolve it for long enough time that it presumably reaches a "typical configuration".
    # Here Swapno suggests to evolve it, starting anywhere, for 1000 seconds, dt = 10e-5 s.

    # Labels for coordinates dictionary
    Y_labels = []
    j = 1
    while j <= dim:
        Y_labels.append(f"X {j}")
        
        j+=1
    
    V_labels = []
    j = 1
    while j <= dim:
        V_labels.append(f"V {j}")
        
        j+=1
    
    
    # Define the times after thermalisation that the coordinates were recorded at.
    times = np.arange(0, simulation_repetitions*delta_t + delta_t*record_steps, delta_t*record_steps)
    
    # Now that the system is thermalised and the thermalised initial conditions are found, evolve the system
    # and run the data collecting simulation.
    recorded_Y1, recorded_Y2, recorded_V1, recorded_V2, recorded_Y3, recorded_Y4, recorded_V3, recorded_V4, recorded_Y5, recorded_Y6, recorded_V5, recorded_V6, recorded_Y7, recorded_Y8, recorded_V7, recorded_V8, recorded_Y9, recorded_V9 = evolving_algorithm(Y_list,
                                                                            V_list, 
                                                                            A_list,
                                          delta_t, record_steps, simulation_repetitions, 
                                          Y_labels, V_labels, N, times)
    
    # Make dataframes out of the dictionaries
    
    # Check if H is conserved

    Y1 = pd.Series(recorded_Y1).T
    Y2 = pd.Series(recorded_Y2).T
    V1 = pd.Series(recorded_V1).T
    V2 = pd.Series(recorded_V2).T
    
    Y3 = pd.Series(recorded_Y3).T
    Y4 = pd.Series(recorded_Y4).T
    V3 = pd.Series(recorded_V3).T
    V4 = pd.Series(recorded_V4).T
    
    Y5 = pd.Series(recorded_Y5).T
    Y6 = pd.Series(recorded_Y6).T
    V5 = pd.Series(recorded_V5).T
    V6 = pd.Series(recorded_V6).T
    
    Y7 = pd.Series(recorded_Y7).T
    Y8 = pd.Series(recorded_Y8).T
    V7 = pd.Series(recorded_V7).T
    V8 = pd.Series(recorded_V8).T
    
    Y9 = pd.Series(recorded_Y9).T
    V9 = pd.Series(recorded_V9).T


    Y = pd.concat([Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9], axis = 1)
    Y = Y.rename(columns = {0:"X1", 1:"X2", 2 : "X3", 3 : "X4", 4 : "X5", 5 : "X6", 6 : "X7", 7 : "X8", 8: "X9"})
    
    V = pd.concat([V1, V2,  X3, X4, X5, X6, X7, X8, X9], axis = 1)
    V = V.rename(columns = {0:"V1", 1:"V2", 2 : "V3", 3 : "V4", 4 : "V5", 5 : "V6", 6 : "V7", 7 : "V8" , 8 : "V9" })
    

    
    return X, V